## Day 1

In [14]:
fo = open("day1.txt")

In [6]:
lines = fo.readlines()

In [10]:
lines = [int(l.replace("\n", "")) for l in lines]

In [12]:
sum(lines)

466

## Day 2

In [30]:
explored = [0]
current = 0
reached_twice = False
while reached_twice == False:
    for freq in lines:
        current += freq
        if current in explored:
            reached_twice = True
            break
        else:
            explored.append(current)

current

987
1973
2959
3945
4931
5917
6903
7889
8875
9861
10847
11833
12819
13805
14791
15777
16763
17749
18735
19721
20707
21693
22679
23665
24651
25637
26623
27609
28595
29581
30567
31553
32539
33525
34511
35497
36483
37469
38455
39441
40427
41413
42399
43385
44371
45357
46343
47329
48315
49301
50287
51273
52259
53245
54231
55217
56203
57189
58175
59161
60147
61133
62119
63105
64091
65077
66063
67049
68035
69021
70007
70993
71979
72965
73951
74937
75923
76909
77895
78881
79867
80853
81839
82825
83811
84797
85783
86769
87755
88741
89727
90713
91699
92685
93671
94657
95643
96629
97615
98601
99587
100573
101559
102545
103531
104517
105503
106489
107475
108461
109447
110433
111419
112405
113391
114377
115363
116349
117335
118321
119307
120293
121279
122265
123251
124237
125223
126209
127195
128181
129167
130153
131139
132125
133111
133576


750

In [33]:
fo2 = open("day2.txt")
box_ids =  [l.replace("\n", "") for l in fo2.readlines()]

In [55]:
def checksum(box_ids):
    count = {2: 0, 3:0}
    for box_id in box_ids:
        fq = defaultdict(int)
        for c in box_id:
            fq[c] += 1
            inv_map = {v: k for k, v in fq.items()}
        if 2 in inv_map:
            count[2] += 1
        if 3 in inv_map:
            count[3] += 1
    rtn = count[2] * count[3]
    return rtn

test_input = ['abcdef', 'bababc', 'abbcde', 'abcccd', 'aabcdd', 'abcdee', 'ababab']

assert checksum(test_input) == 12

In [56]:
checksum(box_ids)

5456

In [58]:
from itertools import combinations, compress

for one, two in combinations(box_ids, 2):    
    diff = [e1 == e2 for e1,e2 in zip(one,two)]    
    if sum(diff) == (len(one) - 1):
        res2 = ''.join(list(compress(one,diff)))
        break
        
res2

'megsdlpulxvinkatfoyzxcbvq'

## Day 3

In [59]:
fo3 = open("day3.txt")
claims =  [l.replace("\n", "") for l in fo3.readlines()]

In [60]:
claims[0]

'#1 @ 912,277: 27x20'

In [66]:
def parse_claim(s):
    identifier, _, dist, size = s.split(' ')
    left, top = map(int, dist[:-1].split(','))
    width, height = map(int, size.split('x'))
    return {"identifier": identifier, "left": left, "top": top, "width": width, "height": height}


In [64]:
def count_overlap(claims):
    parsed_claims = [parse_claim(c) for c in claims]
    
    grid = {i: {k: 0 for k in range(1000)} for i in range(1000)}
    def fill_claim(left, top, width, height):
        for x in range(left, left+width):
            for y in range(top, top+height):
                grid[x][y] += 1
                
    for c in parsed_claims:
        fill_claim(**c)
        
    overlap = 0
    for x in range(1000):
        for y in range(1000):
            if grid[x][y] > 1:
               overlap += 1 
            
    return overlap

count_overlap(claims)

110891

In [78]:
def find_claim_without_overlap(claims):
    parsed_claims = [parse_claim(c) for c in claims]
    
    grid = {i: {k: [] for k in range(1000)} for i in range(1000)}
    def fill_claim(identifier, left, top, width, height):
        for x in range(left, left+width):
            for y in range(top, top+height):
                grid[x][y].append(identifier)
                
    for c in parsed_claims:
        fill_claim(**c)
        
    disqualified = set()
    identifiers = set([c["identifier"] for c in parsed_claims])
    for x in range(1000):
        for y in range(1000):
            if len(grid[x][y]) > 1:
                disqualified.update(grid[x][y])
    return identifiers - disqualified
        
find_claim_without_overlap(claims)


{'#297'}

## Day 4

In [80]:
# sort records chronologically
# tally up sleeping time for all guards
# find guard y with most sleeping time
# find the minute that guard y sleeps the most

fo4 = open("day4.txt")
records =  [l.replace("\n", "") for l in fo4.readlines()]

In [88]:
chronological_records = sorted(records, key=lambda r: (r.split(' ')[0][1:], r.split(' ')[:-1]))
chronological_records[:10]

['[1518-02-05 00:00] Guard #3109 begins shift',
 '[1518-02-05 00:50] falls asleep',
 '[1518-02-05 00:54] wakes up',
 '[1518-02-05 00:57] falls asleep',
 '[1518-02-05 00:58] wakes up',
 '[1518-02-06 00:03] Guard #2411 begins shift',
 '[1518-02-06 00:07] falls asleep',
 '[1518-02-06 00:43] wakes up',
 '[1518-02-06 23:52] Guard #3109 begins shift',
 '[1518-02-07 00:03] falls asleep']

In [122]:
def generate_log(records):
    guard_sleeping_times = {}
    for record in records:
        items = record.split(" ")
        time = int(items[1][3:5])
        if "begins shift" in record:
            guard_no = items[3][1:]
        elif "falls asleep" in record:
            start_time = time
        elif "wakes up" in record:
            if guard_no in guard_sleeping_times:
                guard_sleeping_times[guard_no].append((start_time, time))
            else:
                guard_sleeping_times[guard_no] = [(start_time, time)]
    return guard_sleeping_times

def solution_day4(records):
    logs = generate_log(records)
    laziest_guard_thus_far = 0
    longest_time_slept = 0
    for k, v in logs.items():
        v = sum([i[1] - i[0] for i in v])
        if v > longest_time_slept:
            longest_time_slept = v
            laziest_guard_thus_far = k
    sleeping_times = logs[laziest_guard_thus_far]
    freq_map = {}
    for (s, e) in sleeping_times:
        for i in range(s, e):
            if i in freq_map:
                freq_map[i] += 1
            else:
                freq_map[i] = 1
    inv_map = {v: k for k, v in freq_map.items()}
    return inv_map[max(freq_map.values())] * int(laziest_guard_thus_far)

solution_day4(chronological_records)

101262

In [139]:
def solution_day4_part2(records):
    logs = generate_log(records)
    minutes = {k: {} for k in range(60)}
    for k, v in logs.items():
        for (s, e) in v:
            for i in range(s, e):
                if k in minutes[i]:
                    minutes[i][k] += 1
                else:
                    minutes[i][k] = 1
    
    most_frequent_thus_far = 0
    most_frequent_minute_thus_far = 0
    guard_guilty_of_above_crime = 0
    for minute, freq_map in minutes.items():
        if len(freq_map.values()) == 0:
            continue
        most_frequent = max(freq_map.values())
        inv_map = {v: k for k, v in freq_map.items()}
        if most_frequent > most_frequent_thus_far:
            most_frequent_thus_far = most_frequent
            most_frequent_minute_thus_far = minute
            guard_guilty_of_above_crime = inv_map[most_frequent]
            
    return most_frequent_minute_thus_far * int(guard_guilty_of_above_crime)
            
    
solution_day4_part2(chronological_records)

71976

## Day 5

In [5]:
fo5 = open("day5.txt")
polymer_str =  [l.replace("\n", "") for l in fo5.readlines()][0]

In [39]:
from string import *

def collapse(s):
    p = ['.']
    for u in s:
        last = p[-1]
        if last != u and last.lower() == u.lower():
            p.pop()
        else:
            p.append(u)
    return len(p) - 1

collapse(polymer_str)

9386

In [44]:
from string import ascii_lowercase

freq = {}
for c in ascii_lowercase:
    freq[c] = collapse(polymer_str.replace(c, "").replace(c.upper(), ""))
    
min(freq.values())

4876

## Day 6

In [7]:
fo6 = open("day6.txt")
locations =  [l.replace("\n", "") for l in fo6.readlines()]

In [25]:
coords = [location.split(",") for location in locations]
xvals = [int(coord[0]) for coord in coords]
yvals = [int(coord[1]) for coord in coords]
miny, maxy = min(yvals), max(yvals)
minx, maxx = min(xvals), max(xvals)
grid = {x: {y: "." for y in range(miny-1,maxy+2)} for x in range(minx-1, maxx+2)}

In [26]:
locations = [(int(coord[0]), int(coord[1])) for coord in coords]

def manhattan_distance(coord, locations):
    """
    for a given coordinate with competing locations, return "." if the
    coord is equally far from two or more coordinates or the tuple of the closest
    location as measured by the manhattan distance
    """
    smallest_thus_far = distance_between_tuples(coord, locations[0])
    unique = True
    da_tuple = locations[0]
    for location in locations[1:]:
        distance = distance_between_tuples(coord, location)
        if smallest_thus_far > distance:
            smallest_thus_far = distance
            unique = True
            da_tuple = location
        elif smallest_thus_far == distance:
            unique = False
    if unique == False:
        return "."
    else:
        return da_tuple
        
def distance_between_tuples(tpl1, tpl2):
    return abs(tpl1[0] - tpl2[0]) + abs(tpl1[1] - tpl2[1])

for row_no, rows in grid.items():
    for col_no, point in rows.items():
        grid[row_no][col_no] = manhattan_distance((row_no, col_no), locations)

In [35]:
# all tuples listed as the closest for border squares are disqualified as the

left, right = grid[minx-1].values(), grid[maxx+1].values()
top, bottom = [col[miny-1] for _, col in grid.items()], [col[maxy-1] for _, col in grid.items()]
disqualified = list(set(list(left) + list(right) + top + bottom))

In [46]:
areas = {}
for row_no, rows in grid.items():
    for col_no, point in rows.items():
        winner = grid[row_no][col_no]
        if winner in disqualified:
            continue
        elif winner in areas:
            areas[winner] += 1
        else:
            areas[winner] = 1
            
#inv_map = {v: k for k, v in areas.items()}
max(areas.values())

3620

In [52]:
size = 0

for row_no, rows in grid.items():
    for col_no, point in rows.items():
        winner = grid[row_no][col_no]
        if sum([distance_between_tuples((row_no, col_no), location) for location in locations]) < 10000:
            size += 1
        
size

39930